### Data Cleaning 

create a column that will work with AZ ML Workspace


In [ ]:
# system stuff
import sys
import os

import pandas as pd

# my stuff (abstracted non-important functions)
# Get the project root (one level up from notebooks)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.config import data_path_rvm
from src.prepare_data import create_train_test_dataframes

In [ ]:
# create a single column to use as target for Azure
def create_target(row):
    options = row[1:]
    # find where true
    out = ','.join(['"'+x+'"' for x in options[options=='X'].index])
    if len(out)==0:
        out = '["Other"]'
    else:
        out = '['+out+']'
    return out

In [ ]:
df = pd.read_excel(data_path_rvm, sheet_name = 'Q07a')
df = df.iloc[:, 4:-1]
df.columns = ['Response'] + list(df.columns[1:])
df.head()

In [ ]:
df['target'] = df.apply(create_target, axis=1)
df.head()

In [ ]:
# remove multi-line characters and NA responses
df = df[~pd.isna(df['Response'])]
df['Response'] = df['Response'].apply(lambda x: x.replace('\n',' '))
df.head()

In [ ]:
# split so that it only trains on a subset 
n_train = 1_500
df_train, df_test = create_train_test_dataframes(df, n_train=n_train)

In [ ]:
# pull another 1_000 for validation
df_validate, df_test = create_train_test_dataframes(df_test, n_train=n_train)

In [ ]:
# save these for upload to the workspace
# after removing the int columns
df_train[['Response', 'target']].to_csv(data_path_rvm.replace('.xlsx', '_train.csv'), index=False)
df_validate[['Response', 'target']].to_csv(data_path_rvm.replace('.xlsx', '_validate.csv'), index=False)
df_test[['Response', 'target']].to_csv(data_path_rvm.replace('.xlsx', '_test.csv'), index=False)

In [ ]:
tmp = df_train.iloc[6,:].Response
display(tmp)
tmp.replace('\n',' ')